In [0]:
!pip install ortools

In [0]:
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from __future__ import division
import requests
import json
import urllib



In [0]:
x = requests.get('http://35.239.136.7:8080/bin')
y = x.json()
print(y)

[{u'lat': u'23.2696', u'lng': u'77.3956', u'id': 3}, {u'lat': u'23.2499', u'lng': u'77.4415', u'id': 5}, {u'lat': u'23.2196', u'lng': u'77.3927', u'id': 6}, {u'lat': u'23.2402', u'lng': u'77.5390', u'id': 7}, {u'lat': u'23.0871', u'lng': u'77.5275', u'id': 8}, {u'lat': u'23.2040', u'lng': u'77.4530', u'id': 9}, {u'lat': u'23.1247', u'lng': u'77.4128', u'id': 10}, {u'lat': u'23.3620', u'lng': u'77.3095', u'id': 11}, {u'lat': u'23.2520', u'lng': u'77.4635', u'id': 13}, {u'lat': u'23.3840', u'lng': u'77.4358', u'id': 14}, {u'lat': u'23.1915', u'lng': u'77.4788', u'id': 15}, {u'lat': u'23.2326', u'lng': u'77.3726', u'id': 16}, {u'lat': u'23.2386', u'lng': u'77.4099', u'id': 17}, {u'lat': u'23.2508', u'lng': u'77.4157', u'id': 18}, {u'lat': u'23.2764', u'lng': u'77.3716', u'id': 28}]


In [0]:
coordinates = []
leng = len(y)

for i in range(1,leng):
  coordinates.append(y[i]['lat'])
  coordinates[i-1]+=','+y[i]['lng']
  #coordinates[i-1]= coordinates[i-1]

print(coordinates) 


[u'23.2499,77.4415', u'23.2196,77.3927', u'23.2402,77.5390', u'23.0871,77.5275', u'23.2040,77.4530', u'23.1247,77.4128', u'23.3620,77.3095', u'23.2520,77.4635', u'23.3840,77.4358', u'23.1915,77.4788', u'23.2326,77.3726', u'23.2386,77.4099', u'23.2508,77.4157', u'23.2764,77.3716']


In [0]:
### Finding Distance Matrix ###


def create_data_for_distance_matrix():
  """Creates the data."""
  data = {}
  data['API_key'] = 'API_KEY'
  data['addresses'] = coordinates
                      
  return data

In [0]:
def create_distance_matrix(data):
  addresses = data["addresses"]
  API_key = data["API_key"]
  # Distance Matrix API only accepts 100 elements per request, so get rows in multiple requests.
  max_elements = 100
  num_addresses = len(addresses) # 16 in this example.
  # Maximum number of rows that can be computed per request (6 in this example).
  max_rows = max_elements // num_addresses
  # num_addresses = q * max_rows + r (q = 2 and r = 4 in this example).
  q, r = divmod(num_addresses, max_rows)
  dest_addresses = addresses
  distance_matrix = []
  # Send q requests, returning max_rows rows per request.
  for i in range(q):
    origin_addresses = addresses[i * max_rows: (i + 1) * max_rows]
    response = send_request(origin_addresses, dest_addresses, API_key)
    distance_matrix += build_distance_matrix(response)

  # Get the remaining remaining r rows, if necessary.
  if r > 0:
    origin_addresses = addresses[q * max_rows: q * max_rows + r]
    response = send_request(origin_addresses, dest_addresses, API_key)
    distance_matrix += build_distance_matrix(response)
  return distance_matrix

In [0]:
def send_request(origin_addresses, dest_addresses, API_key):
  """ Build and send request for the given origin and destination addresses."""
  def build_address_str(addresses):
    # Build a pipe-separated string of addresses
    address_str = ''
    for i in range(len(addresses) - 1):
      address_str += addresses[i] + '|'
    address_str += addresses[-1]
    return address_str

  request = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial'
  origin_address_str = build_address_str(origin_addresses)
  dest_address_str = build_address_str(dest_addresses)
  request = request + '&origins=' + origin_address_str + '&destinations=' + \
                       dest_address_str + '&key=' + API_key
  jsonResult = urllib.urlopen(request).read()
  response = json.loads(jsonResult)
  return response

In [0]:
def build_distance_matrix(response):
  distance_matrix = []
  for row in response['rows']:
    row_list = [row['elements'][j]['distance']['value'] for j in range(len(row['elements']))]
    distance_matrix.append(row_list)
  return distance_matrix

In [0]:
data_of_distance_matrix = create_data_for_distance_matrix()
addresses = data_of_distance_matrix['addresses']
API_key = data_of_distance_matrix['API_key']
distance_matrix = create_distance_matrix(data_of_distance_matrix)


  #### Distance Matrix Created ####

In [0]:
#### CVRP Begins ####

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distance_matrix
    data['demands'] = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8,9]##############################
    data['vehicle_capacities'] = [15 for i in range(6)]###############################
    data['num_vehicles'] = 6 ################################
    data['depot'] = 0 #############################
    return data

In [0]:
vehicle_to_route={}
def print_solution(data, manager, routing, assignment):
    """Prints assignment on console."""
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        route_load = 0
        vehicle_to_route[vehicle_id ]=[coordinates[0]]
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            plan_output += ' {0} Load({1}) -> '.format(node_index, route_load)
            vehicle_to_route[vehicle_id ].append(coordinates[node_index])
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += ' {0} Load({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print('Total distance of all routes: {}m'.format(total_distance))
    print('Total load of all routes: {}'.format(total_load))


In [0]:
# Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)


    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    assignment = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if assignment:
        print_solution(data, manager, routing, assignment)
    print(vehicle_to_route)

    final = {}
    final["cor"]=vehicle_to_route


    final = json.dumps(final)
    #print(final)
    loaded = json.loads(final)
    r  = requests.post('http://35.239.136.7:8081/bin',json = loaded)
    #print(vehicle_to_route)
    

Route for vehicle 0:
 0 Load(0) ->  7 Load(8) ->  0 Load(8)
Distance of the route: 5819m
Load of the route: 8

Route for vehicle 1:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 2:
 0 Load(0) ->  0 Load(0)
Distance of the route: 0m
Load of the route: 0

Route for vehicle 3:
 0 Load(0) ->  12 Load(2) ->  6 Load(6) ->  8 Load(14) ->  0 Load(14)
Distance of the route: 70495m
Load of the route: 14

Route for vehicle 4:
 0 Load(0) ->  4 Load(4) ->  5 Load(6) ->  3 Load(8) ->  9 Load(9) ->  2 Load(10) ->  0 Load(10)
Distance of the route: 78689m
Load of the route: 10

Route for vehicle 5:
 0 Load(0) ->  11 Load(1) ->  1 Load(2) ->  10 Load(4) ->  13 Load(8) ->  0 Load(8)
Distance of the route: 37925m
Load of the route: 8

Total distance of all routes: 192928m
Total load of all routes: 40
{0: [u'23.2499,77.4415', u'23.2499,77.4415', u'23.2520,77.4635'], 1: [u'23.2499,77.4415', u'23.2499,77.4415'], 2: [u'23.2499,77.4415', u'23.2499,77.4415'], 3: [u'